In [90]:
from IPython.display import JSON

import requests
import json

url = "https://app.jiuyangongshe.com/jystock-app/api/v1/action/field"

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "application/json",
    "platform": "3",
    "priority": "u=1, i",
    "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors", 
    "sec-fetch-site": "same-site",
    "timestamp": "1733112464730",
    "token": "a57894655cd7b1a4f5665003b2b5140d",
    "cookie": "SESSION=YTZjZmMxYTUtNTYxOS00NzIzLWI3NzktM2QwNTA1ZjIwNzFk; Hm_lvt_58aa18061df7855800f2a1b32d6da7f4=1732592522; Hm_lpvt_58aa18061df7855800f2a1b32d6da7f4=1733112345",
    "Referer": "https://www.jiuyangongshe.com/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
}

payload = {
    "date": "2024-11-29",
    "pc": 1
}

response = requests.post(url, headers=headers, json=payload)

In [78]:
from datetime import datetime
from sqlalchemy import create_engine, Column, String, DateTime, Date, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, sessionmaker

Base = declarative_base()

class StockAction(Base):
    __tablename__ = 'stock_actions'
    
    id = Column(String(50), primary_key=True)
    data = Column(JSON)  # Store all the nested data as JSON
    date = Column(Date)
    name = Column(String)
    create_time = Column(DateTime, default=datetime.now)
    update_time = Column(DateTime)

def init_db():
    engine = create_engine('postgresql://localhost:5432/leek_commune')
    Base.metadata.create_all(engine)
    return sessionmaker(bind=engine)()

# Example usage:
"""
action = StockAction(
    id=str(uuid.uuid4()),
    data={
        'action_field': {
            'id': '49489bfde5eb439e9eabb74c00048611',
            'name': '公告',
            'date': '2024-11-28',
            'status': 0
        },
        'stock': {
            'code': 'sh605136',
            'name': '丽人丽妆'
        },
        # ... other nested data
    }
)
"""


/var/folders/s8/wsqkwqzx6vd6__fhz4sbfh5m0000gp/T/ipykernel_16894/3280364078.py:6: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


"\naction = StockAction(\n    id=str(uuid.uuid4()),\n    data={\n        'action_field': {\n            'id': '49489bfde5eb439e9eabb74c00048611',\n            'name': '公告',\n            'date': '2024-11-28',\n            'status': 0\n        },\n        'stock': {\n            'code': 'sh605136',\n            'name': '丽人丽妆'\n        },\n        # ... other nested data\n    }\n)\n"

In [80]:
session = init_db()

In [96]:
response.json()['data']

[{'date': '2024-11-29',
  'reason': '',
  'action_field_id': '',
  'name': '简图',
  'count': 0},
 {'action_field_id': '42f18424ecf84f4a8e1ccd046955ecf1',
  'name': '公告',
  'date': '2024-11-29',
  'reason': '',
  'status': 0,
  'sort_no': 0,
  'is_delete': '0',
  'delete_time': None,
  'create_time': '2024-11-29 12:17:21',
  'update_time': None,
  'count': 4,
  'list': [{'code': 'sh600589',
    'name': '广东榕泰',
    'article': {'article_id': 'gbepm9jba1',
     'comment_count': 1,
     'like_count': 9,
     'create_time': '2024-11-29 12:17:21',
     'user_id': '1',
     'is_like': 0,
     'action_info': {'article_id': 'gbepm9jba1',
      'action_info_id': 'ed66886f0c2c45d792ba5fbeb09d7ae7',
      'stock_id': 'df19f4460a704c8999b27c07ad278815',
      'action_field_id': '42f18424ecf84f4a8e1ccd046955ecf1',
      'time': '14:12:59',
      'num': '2天2板',
      'price': 547,
      'day': 2,
      'edition': 2,
      'shares_range': 1006.0,
      'reason': None,
      'expound': '拟更名+重整投资人合作华为+合作地

In [92]:
stock_data = response.json()['data']
valid_data = []
print(stock_data[0])
for datum in stock_data:
    item = {}
    item['data'] = datum
    action_field_date = datum.get('date')
    if datum.get('action_field_id') != '':
        exists = session.query(StockAction).filter_by(id=datum.get('action_field_id')).first() != None
        if not exists:
            item['id'] = datum.get('action_field_id')  # Set id to action_field id
            item['date'] = datetime.strptime(action_field_date, '%Y-%m-%d').date()
            valid_data.append(item)

# Bulk insert only valid data
actions = [StockAction(**item) for item in valid_data]
session.bulk_save_objects(actions)
session.commit()

{'date': '2024-11-29', 'reason': '', 'action_field_id': '', 'name': '简图', 'count': 0}
